<a href="https://colab.research.google.com/github/luasm17/LLM_as_a_judge/blob/main/prueba_1_cabuxa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Instalación de dependencias necesarias
# transformers y accelerate se usan para cargar y ejecutar el modelo
# peft y sentencepiece son necesarias para el adapter Cabuxa
!pip install -q transformers accelerate peft sentencepiece

In [2]:
# Imports básicos
# torch se usa para la inferencia con el modelo
# transformers carga el modelo base
# peft permite cargar el adapter Cabuxa
import torch
from transformers import AutoModelForCausalLM, LlamaTokenizer
from peft import PeftModel

In [3]:
# Identificadores del modelo base y del adapter Cabuxa
BASE_MODEL_ID = "huggyllama/llama-7b"
ADAPTER_ID = "irlab-udc/cabuxa-7b"

# Carga del tokenizer del modelo base
# Cabuxa utiliza el tokenizer de LLaMA
tokenizer = LlamaTokenizer.from_pretrained(BASE_MODEL_ID)

# Carga del modelo base LLaMA-7B
# device_map="auto" permite usar GPU si está disponible
base_model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL_ID,
    device_map="auto",
    torch_dtype=torch.bfloat16 if torch.cuda.is_available() else torch.float32
)

# Se carga el adapter Cabuxa-7B sobre el modelo base
model = PeftModel.from_pretrained(base_model, ADAPTER_ID)

# Se pone el modelo en modo evaluación
model.eval()

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/411 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565 - if you loaded a llama tokenizer from a GGUF file you can ignore this message


...

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

adapter_config.json:   0%|          | 0.00/486 [00:00<?, ?B/s]

adapter_model.bin:   0%|          | 0.00/16.8M [00:00<?, ?B/s]

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(32000, 4096, padding_idx=0)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear(
                (base_layer): Linear(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj):

In [10]:
# Función LLM-as-a-Judge para evaluar la concordancia de número
# El prompt y los criterios se mantienen exactamente igual
def cabuxa_judge_concordancia(oracion: str) -> str:
    instruction = f"""
Es un LLM-as-a-judge que vai avaliar un modelo de corrección gramatical en galego.

A túa tarefa é avaliar a saída do modelo de corrección gramatical (GEC)
e decidir se a corrección é ADECUADA con respecto á concordancia de número
(singular/plural entre determinante, substantivo, adxectivo ou verbo).

Devolve a resposta EXACTAMENTE co seguinte formato, sen texto adicional:

output_modelo: ""
etiqueta: <0 ou 1>
explicacion: ""

Criterios:
- etiqueta = 1 → a corrección NON é adecuada (segue habendo erro de concordancia de número)
- etiqueta = 0 → a corrección é adecuada (non hai erro de concordancia de número)
- A explicación debe xustificar só a concordancia de número

Aquí tes un exemplo para orientarte:
- output_modelo: "Os amorodos é vermellos."
- etiqueta: <1>
- explicacion: "O modelo non corrixe o erro de concordancia en "é". "

NON DEBES, BAIXO NINGÚN CONCEPTO, CORRIXIR O OUTPUT DO MODELO QUE TES QUE AVALIAR.
TES QUE LIMITARTE EXCLUSIVAMENTE A DECIDIR SE CONTÉN ERRO DE CONCORDANCIA OU NON.
NON DEBES AVALIAR OUTROS TIPOS DE ERROS.

Agora avalía a seguinte saída dun modelo de GEC:

Oración:
"{oracion}"
""".strip()

def generate_alpaca_prompt(instruction: str) -> str:
    return f"""### Instruction:
{instruction}

### Response:
"""

    # Se construye el prompt final usando el template Alpaca
    prompt = generate_alpaca_prompt(instruction)

    # Tokenización del prompt y envío al dispositivo del modelo
    inputs = tokenizer(prompt, return_tensors="pt")
    inputs = inputs.to(model.device)

    # Generación determinista para evaluación reproducible
    with torch.no_grad():
        output = model.generate(
            **inputs,
            max_new_tokens=64,
            temperature=0.0,
            do_sample=False,
        )

    # Se decodifica únicamente la parte generada por el modelo
    decoded = tokenizer.decode(
        output[0],
        skip_special_tokens=True
    )

    respuesta = decoded[len(prompt):].strip()

    return respuesta

In [9]:
# Oraciones de ejemplo (sin modificar)
oracions = [
    "As decisións tomadas polo comité foron comunicadas aos responsables das distintas áreas.",
    "O grupo de estudantes que participaron no proxecto presentou os resultados finais onte pola tarde.",
    "As propostas que chegaron desde os concellos máis pequenos foi analizadas polo equipo técnico.",
    "A maioría das persoas entrevistadas manifestaron a súa opinión durante a sesión pública."
]

# Ejecución del judge sobre cada oración
for o in oracions:
    print(cabuxa_judge_concordancia(o))
    print()

None

None

None

None

